# 원, 삼각형, 사각형 구분하기
김태영의 컨볼루션신경망모델 만들어보기 [참조]
https://tykimos.github.io/2017/03/08/CNN_Getting_Started/
## 2. 어려운 테스트 문제를 기존 CNN 모델로 푸는 경우

In [4]:
!unzip -qq hard_handwriting_shape.zip -d hard_handwriting_shape

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# seed 값 설정
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터셋 생성하기

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255) #0~1사이의값으로 정규화
#트레인셋을 넘파이 어레이로 만들기
train_generator = train_datagen.flow_from_directory(
    'hard_handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [7]:
!ls hard_handwriting_shape/train
#파일3가지를 만들어진것 확인
#circle	rectangle  triangle

circle	rectangle  triangle


In [8]:
test_datagen = ImageDataGenerator(rescale=1./255) #0~1사이의값으로 정규화
#테스트셋을 넘파이 어레이로 만들기
test_generator = test_datagen.flow_from_directory(
    'hard_handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [9]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2], dtype=int32)

In [10]:
train_generator.filenames[0]

'circle/circle001.png'

### 모델 정의/설정/학습

In [11]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

#### 모델 학습시키기
-첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.  
-steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.  
-epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 50번을 반복적으로 학습.  
-validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 test_generator으로 지정.  
-validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 총 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [13]:
model.fit_generator( # 기존에알고있던것에서 _generator추가됨
        train_generator,
        steps_per_epoch=15,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
15/15 [==============================] - 1s 37ms/step - loss: 1.3894 - accuracy: 0.4825 - val_loss: 1.1747 - val_accuracy: 0.3333
Epoch 2/200
15/15 [==============================] - 0s 22ms/step - loss: 0.5877 - accuracy: 0.8686 - val_loss: 2.2713 - val_accuracy: 0.3333
Epoch 3/200
15/15 [==============================] - 0s 23ms/step - loss: 0.0786 - accuracy: 1.0000 - val_loss: 3.8655 - val_accuracy: 0.3333
Epoch 4/200
15/15 [==============================] - 0s 22ms/step - loss: 0.0211 - accuracy: 1.0000 - val_loss: 4.6669 - val_accuracy: 0.3333
Epoch 5/200
15/15 [==============================] - 0s 22ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 4.9597 - val_accuracy: 0.3333
Epoch 6/200
15/15 [==============================] - 0s 24ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 4.9170 - val_accuracy: 0.3333
Epoch 7/200
15/15 [==============================] - 0s 22ms/step - loss: 4.3962e-04 - accuracy: 1.0000 - val_loss: 5.0794 - val_accuracy: 0.3333
Ep

### 모델평가하기

In [14]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5) #추가됨 _generator
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
#33.33% 이래서 DataAumentation을 해야하는 이유 ㅋ.ㅋ

-- Evaluate --
accuracy: 33.33%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
